In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
n = 40
n_blocks = 4
block_size = int(n/n_blocks)
blocks = [block_size] * n_blocks
probs = [[0.70, 0.05, 0.05, 0.05],
         [0.05, 0.70, 0.05, 0.05],
         [0.05, 0.05, 0.70, 0.05],
         [0.05, 0.05, 0.05, 0.70]]

In [ ]:
strategy_names = [
    'GOT',
#     'got-original-l',
    'L2',
#     'L2-inv',
#     'GOT-original',
    'random',
#     'true'
    'GW',
#     'fGOT',
#     'rrmw',
]
colors = {'GOT': 'red', 'GOT-original': 'red', 'GW': 'green', 'L2': 'blue', 'L2-inv': 'orange', 'random': 'black', 'fGOT': 'yellow',
         'rrmw': 'purple', 'true': 'purple', 'got-original-l': 'yellow'}
results_folder = '../results/07-01/'

p_values = [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

w2_errors = {}
l2_errors = {}
l2_inv_errors = {}
gw_errors = {}
seeds = list(range(0, 25))

empty_seeds = []
for seed in seeds:
    try:
        np.loadtxt(f'{results_folder}/w2_error_got#{seed}.csv')
        np.loadtxt(f'{results_folder}/w2_error_l2#{seed}.csv')
    except (OSError, FileNotFoundError):
        empty_seeds.append(seed)
        
print(f'Remove seeds {empty_seeds}')
for seed in empty_seeds:
    seeds.remove(seed)

for name in strategy_names:
    w2_error_matrix = np.array([np.loadtxt(f'{results_folder}/w2_error_{name}#{seed}.csv') for seed in seeds])
    l2_error_matrix = np.array([np.loadtxt(f'{results_folder}/l2_error_{name}#{seed}.csv') for seed in seeds])
    gw_error_matrix = np.array([np.loadtxt(f'{results_folder}/gw_error_{name}#{seed}.csv') for seed in seeds])
    
    w2_errors[name] = np.mean(w2_error_matrix, axis=0)
    l2_errors[name] = np.mean(l2_error_matrix, axis=0)
    gw_errors[name] = np.mean(gw_error_matrix, axis=0)
    
max_w2_error = np.max(list(w2_errors.values()))
max_l2_error = np.max(list(l2_errors.values()))
max_gw_error = np.max(list(gw_errors.values()))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 2.3), sharey=True)
ax1.set_ylim(0, 1.02)
ax1.set_title('Normalized L2 error')
ax2.set_title('Normalized GOT error')
ax3.set_title('Normalized GW error')
for name in strategy_names:
    l2_error = l2_errors[name] / max_l2_error
    ax1.plot(p_values, l2_error, label=name, marker='*', c=colors[name])
    w2_error = w2_errors[name] / max_w2_error
    ax2.plot(p_values, w2_error, label=name, marker='*', c=colors[name])
    gw_error = gw_errors[name] / max_gw_error
    ax3.plot(p_values, gw_error, label=name, marker='*', c=colors[name])
ax1.legend(loc='lower left')

fig.suptitle(results_folder, fontsize=16, y=1.1)
plt.savefig('../plots/alignment_errors.pdf', bbox_inches='tight')
plt.show()

In [ ]:
import sqlite3
strategies = [
    {'name': 'GOT', 'color': 'red'},
    {'name': 'L2', 'color': 'blue'},
    {'name': 'random', 'color': 'black'},
    {'name': 'gw', 'color': 'green'},
#     {'name': 'fgot', 'color': 'red', 'display_name': 'fGOT', 'marker': '+'},
#     {'name': 'rrmw', 'color': 'purple'},
#     {'name': 'ipfp', 'color': 'orange'},
]

p_values = [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

w2_errors = {}
l2_errors = {}
gw_errors = {}
seeds = list(range(0, 25))

connection = sqlite3.connect(f'../results.db')
cursor = connection.cursor()

for strategy in strategies:
    name = strategy['name']
    w2_errors[name] = []
    l2_errors[name] = []
    gw_errors[name] = []
    for p in p_values:
        cursor.execute(f"SELECT W2_LOSS, L2_LOSS, GW_LOSS FROM alignment WHERE STRATEGY='{name}' and p={p};")
        results = cursor.fetchall()
        assert len(results) > 0, f"No results for strategy '{name}'."
        if len(results) < 25:
            print(f"Only {len(results)} successful seeds for strategy '{name}'.")
        w2_errors[name].append(np.mean(results, axis=0)[0])
        l2_errors[name].append(np.mean(results, axis=0)[1])
        gw_errors[name].append(np.mean(results, axis=0)[2])
cursor.close()
connection.close()

max_w2_error = np.max(list(w2_errors.values()))
max_l2_error = np.max(list(l2_errors.values()))
max_gw_error = np.max(list(gw_errors.values()))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 2.3), sharey=True)
ax1.set_ylim(0, 1.02)
ax1.set_title('Normalized L2 error')
ax1.set_xlabel('Edge removal probability')
ax2.set_title('Normalized GOT error')
ax2.set_xlabel('Edge removal probability')
ax3.set_title('Normalized GW error')
ax3.set_xlabel('Edge removal probability')
for strategy in strategies:
    name = strategy['name']
    display_name = strategy['display_name'] if 'display_name' in strategy else name
    marker = strategy['marker'] if 'marker' in strategy else '*'
    l2_error = l2_errors[name] / max_l2_error
    ax1.plot(p_values, l2_error, label=display_name, marker=marker, c=strategy['color'], lw=1)
    w2_error = w2_errors[name] / max_w2_error
    ax2.plot(p_values, w2_error, label=display_name, marker=marker, c=strategy['color'], lw=1)
    gw_error = gw_errors[name] / max_gw_error
    ax3.plot(p_values, gw_error, label=display_name, marker=marker, c=strategy['color'], lw=1)
ax1.legend(loc='lower left')

# fig.suptitle(results_folder, fontsize=16, y=1.1)
plt.savefig('../plots/alignment_errors.pdf', bbox_inches='tight')
plt.show()